In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
!sudo apt install libopenmpi-dev
!pip install mpi4py

In [2]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 

// funkcja potęgująca bo funkcja w math.h pow nie działa 

double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int numer_procesu;
    int liczba_procesow;
    int tag = 1;
    MPI_Status status;
    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &numer_procesu);
    MPI_Comm_size(MPI_COMM_WORLD, &liczba_procesow);
    clock_t t;
    double time_taken;

    if(numer_procesu == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0;

      for(int i=1; i<liczba_procesow; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        lpi +=x;
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        total_time+=t;
      }
      printf("Wartosc liczby pi = %f \n", 4 * lpi);
      printf("Total time taken = %f \n", total_time);
    }

    if(numer_procesu != 0)
    {
      t = clock();
      double x = potegowanie( -1, numer_procesu - 1) / ( 2 * numer_procesu - 1); 
      printf("Wynik procesu %d = %f \t" , numer_procesu , x);
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      t = clock() - t;
      time_taken = ((double)t)/CLOCKS_PER_SEC;
      MPI_Send(&time_taken, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu nr %d = %f\n",numer_procesu,time_taken);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 1 --allow-run-as-root a.out

Wartosc liczby pi = 0.000000 
Total time taken = 0.000000 
